# Importar bibliotecas

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score, classification_report
from sklearn.preprocessing import MinMaxScaler
import joblib
from sklearn.feature_selection import SelectFromModel

# 1. Carregar o dataset a partir do arquivo 

In [5]:
try:
    df = pd.read_csv('C:\\Users\\alceu\\Documents\\Hackathon\\churninsight-hackathon-one\\data\\raw\\Telco_customer_churn.csv', delimiter=';')
    print("Dataset carregado com sucesso!")
except FileNotFoundError:
    print("Arquivo CSV não encontrado. Verifique o caminho.")
    exit()

Dataset carregado com sucesso!


# 2. Tradução dos nomes das colunas

In [8]:
map_colunas = {
    'CustomerID': 'ID_Cliente', 'Count': 'Contagem', 'Country': 'País', 'State': 'Estado',
    'City': 'Cidade', 'Zip Code': 'CEP', 'Lat Long': 'Lat_Long', 'Latitude': 'Latitude',
    'Longitude': 'Longitude', 'Gender': 'Gênero', 'Senior Citizen': 'Idoso', 'Partner': 'Parceiro',
    'Dependents': 'Dependentes', 'Tenure Months': 'Tempo_Contrato', 'Phone Service': 'Serviço_Telefônico',
    'Multiple Lines': 'Múltiplas_Linhas', 'Internet Service': 'Serviço_Internet',
    'Online Security': 'Segurança_Online', 'Online Backup': 'Backup_Online',
    'Device Protection': 'Proteção_Dispositivo', 'Tech Support': 'Suporte_Técnico',
    'Streaming TV': 'Streaming_TV', 'Streaming Movies': 'Streaming_Filmes', 'Contract': 'Contrato',
    'Paperless Billing': 'Fatura_Online', 'Payment Method': 'Método_Pagamento',
    'Monthly Charges': 'Cobranças_Mensais', 'Total Charges': 'Cobranças_Totais',
    'Churn Label': 'Churn_Label', 'Churn Value': 'Churn_Valor', 'Churn Score': 'Churn Score',
    'CLTV': 'CLTV', 'Churn Reason': 'Churn_Motivo'
}

In [10]:
# Renomear as colunas
df.rename(columns=map_colunas, inplace=True)

# 3. Pré-processamento dos dados

In [13]:
# Transformar Cobranças_Totais para numérico
df['Cobranças_Totais'] = pd.to_numeric(df['Cobranças_Totais'], errors='coerce')

In [15]:
# Preencher valores ausentes da coluna 'Cobranças_Totais' com a média
df['Cobranças_Totais'] = df['Cobranças_Totais'].fillna(df['Cobranças_Totais'].mean())

In [17]:
# Transformar a coluna Churn para numérico
df['Churn'] = df['Churn_Valor']

In [19]:
# Encoding das colunas categóricas (One-Hot Encoding)
colunas_categoricas = df.select_dtypes(include=['object']).columns.tolist()

In [21]:
# Garante que 'ID_Cliente' não seja incluída no encoding
if 'ID_Cliente' in colunas_categoricas:
    colunas_categoricas.remove('ID_Cliente')

In [23]:
df = pd.get_dummies(df, columns=colunas_categoricas, drop_first=True)

# 3. Modelagem

In [54]:
# Remover 'ID_Cliente' e colunas desnecessárias antes da modelagem
# Remoção de colunas "problemáticas"
colunas_a_remover = ['ID_Cliente', 'Churn_Label', 'Churn_Motivo', 'Churn_Score', 'CLTV']
df = df.drop(colunas_a_remover, axis=1, errors='ignore')

In [55]:
# Definir as features (X) e a variável alvo (y)
X = df.drop('Churn', axis=1, errors = 'ignore')
y = df['Churn']

In [68]:
# Dividir os dados em conjuntos de treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=123) # Testar com uma semente diferente

In [70]:
# Escalonamento das features numéricas
scaler = MinMaxScaler()
X_treino = scaler.fit_transform(X_treino)
X_teste = scaler.transform(X_teste)

In [72]:
# Testar Regressão Logística
modelo_lr = LogisticRegression(random_state=42, solver='liblinear', penalty='l1', C=0.1) # Testando um modelo mais simples
modelo_lr.fit(X_treino, y_treino)
y_pred_lr = modelo_lr.predict(X_teste)

In [74]:
print("\nRegressão Logística:")
print(f"Acurácia: {accuracy_score(y_teste, y_pred_lr):.4f}")
print(f"F1-score: {f1_score(y_teste, y_pred_lr):.4f}")
print(f"Recall: {recall_score(y_teste, y_pred_lr):.4f}")
print(f"AUC: {roc_auc_score(y_teste, y_pred_lr):.4f}")
print(classification_report(y_teste, y_pred_lr))


Regressão Logística:
Acurácia: 1.0000
F1-score: 1.0000
Recall: 1.0000
AUC: 1.0000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1028
           1       1.00      1.00      1.00       381

    accuracy                           1.00      1409
   macro avg       1.00      1.00      1.00      1409
weighted avg       1.00      1.00      1.00      1409



# 4. Serialização do Modelo

In [77]:
# Escolher o melhor modelo (com base nas métricas)
melhor_modelo = modelo_lr

In [79]:
# Salvar o modelo
joblib.dump(melhor_modelo, 'modelo_churn.joblib')
print("Modelo serializado e salvo como 'modelo_churn.joblib'")

Modelo serializado e salvo como 'modelo_churn.joblib'


# Relatório de Análise de Churn de Clientes 

## Dataset:
 Nome: Telco Customer Churn
 Fonte: Kaggle

## Problemas:
Overfitting persistente, mesmo após aplicar técnicas de regularização e seleção de features.
Após a limpeza e o pré-processamento, o dataset ficou com pouca variabilidade nos dados, impossibilitando a criação de um modelo preditivo generalizável.

## Etapas Realizadas:
 Extração de Dados: Os dados foram carregados de um arquivo CSV.
Tradução dos Nomes das Colunas: Os nomes das colunas foram traduzidos para português para facilitar a interpretação.
 Pré-processamento dos Dados:
A coluna 'TotalCharges' foi convertida para o tipo numérico e os valores ausentes foram preenchidos com a média.
A coluna 'Churn' foi convertida para o tipo numérico (0 e 1).
As colunas categóricas foram codificadas usando one-hot encoding.
 Modelagem:
Vários modelos de classificação foram testados, incluindo Regressão Logística e Gradient Boosting Machine.
Técnicas de regularização foram aplicadas para tentar mitigar o overfitting.
A seleção de features foi realizada usando o SelectFromModel.
 Avaliação:
A validação cruzada estratificada foi usada para avaliar o desempenho do modelo.
Acurácia, precisão, recall e F1-score foram usados como métricas de avaliação.

### Resultados:
Apesar dos esforços para mitigar o overfitting, todos os modelos testados apresentaram um desempenho perfeito (1.00 em todas as métricas) nos dados de teste. Isso indica que o modelo estava memorizando os dados de treino em vez de aprender os padrões subjacentes, o que tornava impossível obter uma estimativa confiável do seu desempenho em dados novos.

### Conclusão:
O conjunto de dados utilizado neste estudo apresentou um problema significativo de overfitting que não pôde ser resolvido com as técnicas de modelagem e pré-processamento aplicadas. A causa mais provável é a falta de complexidade nos dados, o que permitiu que os modelos aprendessem os padrões de cor em vez de generalizar para dados novos.

### Recomendações:
Obter um novo conjunto de dados: É necessário encontrar um novo conjunto de dados que seja mais complexo e representativo do problema de churn de clientes.